In [6]:
import numpy as np
import optuna

最もシンプルな使い方

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/001_first.html

In [2]:
# 目的関数の定義
def objective(trial):
    # -10から10までの値を取るfloat変数の定義（最適化対象のパラメタ）
    # hyperoptと異なるのはオラクルたる目的関数側で変数定義をやる点？
    x = trial.suggest_float("x", -10, 10)

    return (x - 2) ** 2

上の例では`float`型のパラメタを扱ったがそれ以外にも整数値、カテゴリ変数など色々使える。探索空間の指定の仕方等、詳しくは下記URLを参照

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/002_configurations.html

In [3]:
# 最適化プロジェクトの定義
study = optuna.create_study()
# 最適化実行
study.optimize(objective, n_trials=100)

[I 2022-06-18 01:05:22,958] A new study created in memory with name: no-name-2fbd79a2-6120-4af1-90f3-c437c2697b1d
[I 2022-06-18 01:05:22,960] Trial 0 finished with value: 41.22795321702103 and parameters: {'x': -4.420899720212194}. Best is trial 0 with value: 41.22795321702103.
[I 2022-06-18 01:05:22,961] Trial 1 finished with value: 21.48037042085271 and parameters: {'x': -2.6346920524294504}. Best is trial 1 with value: 21.48037042085271.
[I 2022-06-18 01:05:22,962] Trial 2 finished with value: 34.42950383392392 and parameters: {'x': -3.867665961344759}. Best is trial 1 with value: 21.48037042085271.
[I 2022-06-18 01:05:22,962] Trial 3 finished with value: 10.468880693997265 and parameters: {'x': -1.2355649729216172}. Best is trial 3 with value: 10.468880693997265.
[I 2022-06-18 01:05:22,963] Trial 4 finished with value: 59.54234906384511 and parameters: {'x': -5.716368904079503}. Best is trial 3 with value: 10.468880693997265.
[I 2022-06-18 01:05:22,964] Trial 5 finished with value:

In [4]:
# 最小値の表示
best_params = study.best_params
found_x = best_params["x"]
print("Found x: {}, (x - 2)^2: {}".format(found_x, (found_x - 2) ** 2))

Found x: 1.9751052980128248, (x - 2)^2: 0.0006197461870302674


探索の経過の可視化 (**Plotlyパッケージをインストール必要**)

https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html

In [5]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

バッチ最適化

基本的にはAsk-and-Tellインターフェースという対話的な利用の一ユースケースとしてバッチ最適化がある。たとえば、観山が困っていたパラメタ側に関する制約を守らせるためのサンプリングを事前にして、満たす入力値とこれに対する評価値のみをOptunaに伝えたいときもおそらくこの対話インターフェースを使えばよさそう。

https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/009_ask_and_tell.html

In [8]:
# バッチでパラメタを受け取って値を返す目的関数
# バッチ最適化というか対話的にOptunaを使う場合には目的関数内でパラメタを定義しないみたい
def batched_objective(xs: np.ndarray, ys: np.ndarray):
    return xs ** 2 + ys

In [12]:
# バッチサイズ
batch_size = 10
# 問い合わせ回数
num_query = 10
study = optuna.create_study()

for _ in range(num_query):
    # create batch
    trial_ids = []
    x_batch = []
    y_batch = []

    # バッチサイズ分だけデータを作る
    for _ in range(batch_size):
        # Optunaにぶち込むためのデータ形式を作る
        trial = study.ask()
        trial_ids.append(trial.number)
        # ここでパラメタの定義と探索空間を指定
        x_batch.append(trial.suggest_float("x", -10, 10))
        y_batch.append(trial.suggest_float("y", -10, 10))

    # 現在のバッチに対する評価値をオラクルに問い合わせる
    x_batch = np.array(x_batch)
    y_batch = np.array(y_batch)
    objectives = batched_objective(x_batch, y_batch)

    # 問い合わせ結果をOptunaに伝える
    for trial_id, objective in zip(trial_ids, objectives):
        study.tell(trial_id, objective)

[I 2022-06-18 01:18:09,471] A new study created in memory with name: no-name-2a25ea9f-9f82-4f40-b035-213fdae31a15


In [14]:
# 10個ずつ10回聞いたのでトライアル合計数としては100個分となる
# 割と1回の問い合わせバッチの中身が偏っている印象を受けるがアルゴリズムの選択を変えると挙動が変わるのかもしれない
plot_optimization_history(study)